# 1.导入相关库

In [1]:
import numpy as np
import xlrd
import copy

# 2.设置基本信息/读取Excel数据

In [2]:
Number = 402
Week = 240
InitRow = 1
InitColumn = 2

Require = np.zeros([Number , Week])
Supply = np.zeros([Number , Week])
filename = "附件1 近5年402家供应商的相关数据.xls"
file = xlrd.open_workbook(filename)
SheetName = file.sheet_names()
Sheet1 = file.sheet_by_name(sheet_name = SheetName[0])
Sheet2 = file.sheet_by_name(sheet_name = SheetName[1])
for j in range(Number):
    for i in range(Week):
        Require[j][i] = Sheet1.cell_value(rowx = j+InitRow , colx = i+InitColumn)
        Supply[j][i] = Sheet2.cell_value(rowx = j+InitRow , colx = i+InitColumn)
        pass
    pass

# 3.数据处理获得四个值

假设有$m$个供应商，第j个供应商的四个参数值

其中用到了$n=240$周(共五年)的数据，用来索引天数

$r$指require，订货量
$s$指供应量supply

$r_{ji}$表示厂家在第i周，向j号供应商提交的订单数
$s_{ji}$表示j号供应商，在第i周提供给厂家的货物

$$
\begin{aligned}
\begin{cases}
u_{j.1} = \frac{\sum_{i = 1}^n s_{ji} - \sum_{i = 1}^n r_{ji}}{\sum_{i = 1}^n r_{ji}}\\
u_{j.2} = \sum_{i = 1}^n s_{ji}\\
u_{j.3} = \frac{\sum_{i = 1}^n r_{ji}\cdot s_{ji}}{\sqrt{ (\sum_{i = 1}^j r_{ji}^2)\cdot (\sum_{i = 1}^j s_{ji}^2)}}\\
u_{j.4} = \sum_{i = 1}^n Count(j , i)
\end{cases}
\end{aligned}
$$

其中

$$
\begin{aligned}

Count(j , i) = 
\begin{cases}
0 ,& \sum_{k = 1}^i r_{jk} > \sum_{k = 1}^i s_{jk}\\

1 ,& \sum_{k = 1}^i r_{jk} < \sum_{k = 1}^i s_{jk}
\end{cases}

\end{aligned}
$$

In [3]:
class Supplier:

    def __init__(self , r , s):
        self.GetU1(r , s)
        self.GetU2(r , s)
        self.GetU3(r , s)
        self.GetU4(r , s)
        pass

    def GetU1(self , r , s):
        percent = (s.sum() - r.sum()) / r.sum()
        self.u1 = percent
        pass

    def GetU2(self , r , s):
        Sum = s.sum()
        self.u2 = Sum
        pass

    def GetU3(self , r , s):
        a = r.dot(s)
        b = np.sqrt(r.dot(r) * s.dot(s))
        self.u3 = a/b
        pass

    def GetU4(self , r , s):
        vec01 = np.zeros(r.size)
        for i in range(r.size):
            if r[0:i+1].sum() < s[0:i+1].sum():
                vec01[i] = 1
                pass
            pass
        self.u4 = vec01.sum()
        pass
    pass

ListSupplier = list()
for i in range(Number):
    r = Require[i]
    s = Supply[i]
    item = Supplier(r , s)
    ListSupplier.append(item)
    pass

U = np.zeros([Number , 4])
for j in range(Number):
    item = ListSupplier[j]
    U[j][0] = item.u1
    U[j][1] = item.u2
    U[j][2] = item.u3
    U[j][3] = item.u4
    pass

# 4.数据归一化

$$
\bar{u}_{j,l} = \frac{u_{j,l} - u_{\min , l}}{u_{\max,l} - u_{\min,l}},l=1,2,3,4
$$

In [4]:
def Normalize(vec):
    return (vec - vec.min()) / (vec.max() - vec.min())
    pass

Un = copy.deepcopy(U.T)
for i in range(4):
    Un[i] = Normalize(Un[i])
    pass

# 5.比重计算

$$
p_{jl} = \frac{\bar{u}_{j , l}}{\sum_{j = 1}^n \bar{u}_{j,l}},l=1,2,3,4
$$

In [5]:
def GetP(X):
    M , N = X.shape
    P = np.zeros([M , N])
    for j in range(M):
        P[j] = X[j] / X[j].sum()
        pass
    return P
    pass
Pn = GetP(Un)

# 6.信息熵及权值计算

$$
e_l = -\frac{1}{\ln n}\sum_{j = 1}^n p_{jl}\cdot \ln p_{jl},l=1,2,3,4\\

d_l = 1-e_l , l=1,2,3,4\\
\omega_l = \frac{d_l}{\sum_{l=1}^4 d_l}
$$

In [6]:
def GetE(P , m):
    k = 1. / np.log(m)
    M , N = P.shape
    E = np.zeros(M)
    for j in range(M):
        SumP = 0.
        for i in range(N):
            p = P[j][i]
            SumP += p * np.log(p)
            pass
        E[j] = -k * SumP
        pass
    return E
    pass

En = GetE(Pn+1e-20 , Number) #修正!!
Dn = 1.-En
Wn = Dn / Dn.sum()
print(Wn)

[0.08390638 0.55491882 0.0862727  0.2749021 ]


# 7.计算综合指标

$$
F_j = \sum_{l = 1}^4 \omega_l \cdot p_{jl} 
$$

In [7]:
Fn = np.zeros(Number)
for j in range(Number):
    Fj = 0.
    for i in range(4):
        Fj += Wn[i] * Pn[i][j]
        pass
    Fn[j] = Fj
    pass
Fn

array([1.46247418e-04, 9.77024845e-04, 2.64455956e-03, 8.72424111e-05,
       4.95993010e-03, 4.03417528e-05, 2.91345194e-03, 4.46632131e-04,
       2.61284235e-05, 2.57393069e-04, 3.24070019e-03, 6.96237232e-05,
       3.72171425e-03, 1.09625569e-03, 1.10854683e-07, 1.07829064e-03,
       6.18758131e-04, 4.96838046e-04, 4.06617744e-04, 3.67000002e-04,
       7.71139006e-04, 4.37082481e-04, 3.95510287e-04, 3.81562794e-04,
       4.03564097e-03, 3.97969944e-03, 1.91896391e-03, 6.02941205e-04,
       6.08721292e-05, 4.06921079e-03, 8.98305074e-03, 6.78388683e-04,
       1.78677695e-03, 2.84545466e-05, 5.73090936e-04, 4.02507801e-03,
       8.09076663e-03, 4.46879765e-04, 4.01449606e-04, 5.23853242e-03,
       2.23769324e-04, 3.87830368e-04, 6.84883297e-05, 3.38165508e-03,
       1.73348967e-04, 1.65599387e-03, 8.48291235e-06, 5.51325998e-04,
       6.37544890e-04, 3.66068854e-03, 5.41851382e-06, 1.11430347e-03,
       3.96369551e-03, 6.31754023e-04, 4.44459578e-03, 5.34354195e-04,
      

# 8.寻找Top50

In [8]:
Pos = np.argsort(1-Fn)
Pos = Pos[0:50] + 1
Pos

array([229, 361, 140, 108, 151, 282, 275, 340, 329, 139, 268, 306, 131,
       330, 308, 356, 194, 348, 352, 143, 247, 201, 395, 307, 374, 284,
       365,  31,  37, 126, 294, 218,  80,  40,   5, 266, 367, 364, 189,
       338, 123,  55, 169, 221, 174, 324, 113, 237, 141, 175])